In [154]:
import base64
import hashlib
import html
import json
import os
import re
import urllib.parse
import requests

from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs


In [155]:
client_id = "ch"
username = "123"
password = "123"
redirect_uri = "https://oauth.pstmn.io/v1/callback"
state="123"


In [156]:
code_verifier = base64.urlsafe_b64encode(os.urandom(40)).decode('utf-8')
code_verifier = re.sub('[^a-zA-Z0-9]+', '', code_verifier)
code_verifier, len(code_verifier)

('IfonahjMLJsfoR8PmsfJsKi1VlvQ2rR3iWBAGHpLz0RTPyucmQ', 50)

In [157]:
code_challenge = hashlib.sha256(code_verifier.encode('utf-8')).digest()
code_challenge = base64.urlsafe_b64encode(code_challenge).decode('utf-8')
code_challenge = code_challenge.replace('=', '')
code_challenge, len(code_challenge)

('BSa2p9V7je5FUGhKO6dah33g0-VbEgjqUkgyousZUQY', 43)

In [158]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs


# Simulate the request to get the HTML response
url = "http://localhost:8080/realms/htc/protocol/openid-connect/auth"  # Replace with your actual URL
params={
        "response_type": "code",
        "client_id": client_id,
        "state": state,
        "scope": "openid",
        "redirect_uri": redirect_uri,
        "code_challenge": code_challenge,
        "code_challenge_method": "S256",
        "response_mode" : "fragment",
    }

response = requests.get(url, params=params)

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Extract the form's action attribute
form_action_url = soup.find('form', {'id': 'kc-form-login'})['action']

# Parse the URL to extract query parameters
parsed_url = urlparse(form_action_url)
query_params = parse_qs(parsed_url.query)

# Extract specific values
session_code = query_params.get('session_code', [None])[0]
execution = query_params.get('execution', [None])[0]
client_id = query_params.get('client_id', [None])[0]
tab_id = query_params.get('tab_id', [None])[0]
client_data = query_params.get('client_data', [None])[0]

# Print extracted values
print("session_code:", session_code)
print("execution:", execution)
print("client_id:", client_id)
print("tab_id:", tab_id)
print("client_data:", client_data)


session_code: G2rRXDCOwOctgBG4NmHTXZoE7i3nbzmw7xNnsKeU_wM
execution: 120d8b9b-2384-4c29-86d9-4171413f2b1c
client_id: ch
tab_id: 3PaDmS0ei1M
client_data: eyJydSI6Imh0dHBzOi8vb2F1dGgucHN0bW4uaW8vdjEvY2FsbGJhY2siLCJydCI6ImNvZGUiLCJybSI6ImZyYWdtZW50Iiwic3QiOiIxMjMifQ


In [159]:
form_action_url

'http://localhost:8080/realms/htc/login-actions/authenticate?session_code=G2rRXDCOwOctgBG4NmHTXZoE7i3nbzmw7xNnsKeU_wM&execution=120d8b9b-2384-4c29-86d9-4171413f2b1c&client_id=ch&tab_id=3PaDmS0ei1M&client_data=eyJydSI6Imh0dHBzOi8vb2F1dGgucHN0bW4uaW8vdjEvY2FsbGJhY2siLCJydCI6ImNvZGUiLCJybSI6ImZyYWdtZW50Iiwic3QiOiIxMjMifQ'

In [160]:
# Dont need right now


page = response.text
form_action = html.unescape(re.search(r'<form\s+.*?\s+action="(.*?)"', page, re.DOTALL).group(1))
form_action
#"http://localhost:8080/realms/htc/protocol/openid-connect/auth?response_type=code&client_id=ch&state=123&scope=openid%20profile%20channel_id_scope&redirect_uri=https%3A%2F%2Foauth.pstmn.io%2Fv1%2Fcallback&code_challenge=EBmvYVRAlV1TtWWYm7d_nsfTC6QhTxVTvCsyhkkPJeI&code_challenge_method=S256&X-Custom-Header=expected-value&mob=123&mob1=123"

# Parse the URL
parsed_url = urllib.parse.urlparse(form_action)

# Parse the query parameters
params = urllib.parse.parse_qs(parsed_url.query)

# Decode and print each parameter
for key, value in params.items():
    # value is a list because there can be multiple values for a key
    print(f"{key}: {value[0]}")

# If you want a dictionary with single values:
decoded_params = {key: value[0] for key, value in params.items()}

print("\nDecoded parameters as dictionary:")
print(decoded_params)


session_code: G2rRXDCOwOctgBG4NmHTXZoE7i3nbzmw7xNnsKeU_wM
execution: 120d8b9b-2384-4c29-86d9-4171413f2b1c
client_id: ch
tab_id: 3PaDmS0ei1M
client_data: eyJydSI6Imh0dHBzOi8vb2F1dGgucHN0bW4uaW8vdjEvY2FsbGJhY2siLCJydCI6ImNvZGUiLCJybSI6ImZyYWdtZW50Iiwic3QiOiIxMjMifQ

Decoded parameters as dictionary:
{'session_code': 'G2rRXDCOwOctgBG4NmHTXZoE7i3nbzmw7xNnsKeU_wM', 'execution': '120d8b9b-2384-4c29-86d9-4171413f2b1c', 'client_id': 'ch', 'tab_id': '3PaDmS0ei1M', 'client_data': 'eyJydSI6Imh0dHBzOi8vb2F1dGgucHN0bW4uaW8vdjEvY2FsbGJhY2siLCJydCI6ImNvZGUiLCJybSI6ImZyYWdtZW50Iiwic3QiOiIxMjMifQ'}


In [161]:
form_action

'http://localhost:8080/realms/htc/login-actions/authenticate?session_code=G2rRXDCOwOctgBG4NmHTXZoE7i3nbzmw7xNnsKeU_wM&execution=120d8b9b-2384-4c29-86d9-4171413f2b1c&client_id=ch&tab_id=3PaDmS0ei1M&client_data=eyJydSI6Imh0dHBzOi8vb2F1dGgucHN0bW4uaW8vdjEvY2FsbGJhY2siLCJydCI6ImNvZGUiLCJybSI6ImZyYWdtZW50Iiwic3QiOiIxMjMifQ'

In [162]:
cookie = response.headers['Set-Cookie']
cookie = '; '.join(c.split(';')[0] for c in cookie.split(', '))
cookie

'AUTH_SESSION_ID=47794425-a72d-4556-adf2-5786459aafb9; AUTH_SESSION_ID_LEGACY=47794425-a72d-4556-adf2-5786459aafb9; KC_RESTART=eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..n8oOSmHaF1siLGufrzg2Qg.TXeFcEUcArm2dD5AuCfLVF6nmNm3bcqj5_A-tyg5W-O8fHjGkNOfzDSrsc1tQN3ePUxj7eI1B_wkLy4b93yBLoAOR5BOkl_laJuic3NvnwyhlVlmWTR1BCVA2KnBCyIfZj4ciPMCh6RRdQBfYW7fcygIxKdte1vnVpC2LhcMKp_C9oPNpLbZKeNoNe3zBYkV3teNVc8Egm8DGdT0a6lTA9Tf8eWsBu7SOKWiVjcHbOJMS7kkL_ZiGNCO08cUhhbTm6XXLGvh09sGyNHfUyUledpr6jICc4bVWrd7gmyeUDzkP1400FCuEXy4qSEUbgBfAvBjfB_L9OhseI8DH0UrLAC9O8SP1OnadIeLebHVxt9Kc28u7IyDHn3NR2Rigputoyc0OSeGXq1D8t23lQAWRbHaEztaggogejDN5sBE-QE4A_29ACLtNKRXePsfiC-sUfonoyqUm4hPuxUTrJ-QPLa6s6l0WeW5UZcEH94UOY5WUMENhoTHOVJOpe-4AXsrFnAkWaahucC6yIRxDiVIZf0zxqIWA5WeXRMzdyiSbH7-lAijZg11saLe2ruHFmKbomU88gRn8s79EC2-kzZA6Htyp7LtSdQfv-zQJoPf4xRG7VQa6u-e8bIP6Kp4MPa6D6M0hWwsEmHnQvePEAG1eT32l7wbQpxoAcvBIzlpWdcmv1q8KMOaCCbnfCRwJ4xEVHrJneXuixhqAEWGuB5lxIte_nYa0Lenj6S8IJp69RkEWF9QclPeI-Dc84QQu0A0-RUT5JLgRA5kYgPiCp6tQjJliuhjWZiP8

In [163]:
import requests

# Extracted values from previous steps
session_code = decoded_params.get('session_code')
execution = decoded_params.get('execution')
client_id = decoded_params.get('client_id')
tab_id = decoded_params.get('tab_id')
client_data = decoded_params.get('client_data')

# URL to send the POST request
url = 'http://localhost:8080/realms/fetchusers/login-actions/authenticate'

# Headers for the request
headers = {
    "Upgrade-Insecure-Requests": "1",
    "Origin": "null",
    "Content-Type": "application/x-www-form-urlencoded",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Postman/11.10.0 Electron/28.3.1 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-User": "?1",
    "Sec-Fetch-Dest": "document",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US",
    "Cookie": cookie  # Make sure this is valid
}

# Data to send in the POST request
data = {
    'session_code': session_code,
    'execution': execution,
    'client_id': client_id,
    'tab_id': tab_id,
    'client_data': client_data,
    'username': '123',  # Replace with actual username if needed
    'password': '123'   # Replace with actual password if needed
}

# Send the POST request
response = requests.post(url, headers=headers, data=data)

# Print the response content and status code
print(f"Status Code: {response.status_code}")
print(f"Response Text: {response.text}")
print(f"Response Headers: {response.headers}")


Status Code: 400
Response Text: <!DOCTYPE html>
<html class="login-pf">

<head>
    <meta charset="utf-8">
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
    <meta name="robots" content="noindex, nofollow">

            <meta name="viewport" content="width=device-width,initial-scale=1"/>
    <title>Sign in to fetchusers</title>
    <link rel="icon" href="/resources/ivzwa/login/keycloak/img/favicon.ico" />
            <link href="/resources/ivzwa/common/keycloak/node_modules/@patternfly/patternfly/patternfly.min.css" rel="stylesheet" />
            <link href="/resources/ivzwa/common/keycloak/node_modules/patternfly/dist/css/patternfly.min.css" rel="stylesheet" />
            <link href="/resources/ivzwa/common/keycloak/node_modules/patternfly/dist/css/patternfly-additions.min.css" rel="stylesheet" />
            <link href="/resources/ivzwa/common/keycloak/lib/pficon/pficon.css" rel="stylesheet" />
            <link href="/resources/ivzwa/login/keycloak/css/l

In [164]:
import requests

url = "http://localhost:8080/realms/htc/login-actions/authenticate"
params = {
    "session_code": decoded_params.get('session_code'),
    "execution": decoded_params.get('execution'),
    "client_id": decoded_params.get('client_id'),
    "tab_id": decoded_params.get('tab_id'),
    "client_data": decoded_params.get('client_data'),
    "username" :"123",
    "password" :"123",
}

response = requests.get(url, params=params)

print("Status Code:", response.status_code)
print("Response Body:", response.text)


Status Code: 400
Response Body: <!DOCTYPE html>
<html class="login-pf">

<head>
    <meta charset="utf-8">
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
    <meta name="robots" content="noindex, nofollow">

            <meta name="viewport" content="width=device-width,initial-scale=1"/>
    <title>Sign in to htc</title>
    <link rel="icon" href="/resources/ivzwa/login/keycloak/img/favicon.ico" />
            <link href="/resources/ivzwa/common/keycloak/node_modules/@patternfly/patternfly/patternfly.min.css" rel="stylesheet" />
            <link href="/resources/ivzwa/common/keycloak/node_modules/patternfly/dist/css/patternfly.min.css" rel="stylesheet" />
            <link href="/resources/ivzwa/common/keycloak/node_modules/patternfly/dist/css/patternfly-additions.min.css" rel="stylesheet" />
            <link href="/resources/ivzwa/common/keycloak/lib/pficon/pficon.css" rel="stylesheet" />
            <link href="/resources/ivzwa/login/keycloak/css/login.cs

In [165]:
import requests

# Create a session object to handle cookies
session = requests.Session()

# Define the URL and parameters
url = "http://localhost:8080/realms/htc/login-actions/authenticate"
params = {
    "session_code": "QhwuvC0SqvgtIvBtghD62_RBGQ93XS1GysPPUtM4BZQ",
    "execution": "120d8b9b-2384-4c29-86d9-4171413f2b1c",
    "client_id": "ch",
    "tab_id": "1ayLWkwq4Pg",
    "client_data": "eyJydSI6Imh0dHBzOi8vb2F1dGgucHN0bW4uaW8vdjEvY2FsbGJhY2siLCJydCI6ImNvZGUiLCJzdCI6IjEyMyJ9"
}

# Define body data for POST request
data = {
    "username": "123",
    "password": "123"
}

# Define headers, if needed (e.g., to mimic a browser)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'application/json',
    'cookie' : cookie,
    "session_code": decoded_params.get('session_code'),
    "execution": decoded_params.get('execution'),
    "client_id": decoded_params.get('client_id'),
    "tab_id": decoded_params.get('tab_id'),
    "client_data": decoded_params.get('client_data'),
}

data = {
    "username" :"123",
    "password" :"123",
}

# Send the POST request with the session object
response = session.post(url, params=params, data=data, headers=headers)

# Print the status code and response body
print("Status Code:", response.status_code)
print("Response Body:", response.text)


Status Code: 400
Response Body: {"error":"Cookie not found. Please make sure cookies are enabled in your browser."}


In [ ]:
import requests

# Create a session object to handle cookies
session = requests.Session()

# Define the URL and parameters
url = "http://localhost:8080/realms/htc/login-actions/authenticate"
params = {
    "session_code": "QhwuvC0SqvgtIvBtghD62_RBGQ93XS1GysPPUtM4BZQ",
    "execution": "120d8b9b-2384-4c29-86d9-4171413f2b1c",
    "client_id": "ch",
    "tab_id": "1ayLWkwq4Pg",
    "client_data": "eyJydSI6Imh0dHBzOi8vb2F1dGgucHN0bW4uaW8vdjEvY2FsbGJhY2siLCJydCI6ImNvZGUiLCJzdCI6IjEyMyJ9"
}

# Define body data for POST request
data = {
    "username": "123",
    "password": "123"
}

# Define headers, if needed (e.g., to mimic a browser)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'application/json',
    'cookie' : cookie,
    "session_code": decoded_params.get('session_code'),
    "execution": decoded_params.get('execution'),
    "client_id": decoded_params.get('client_id'),
    "tab_id": decoded_params.get('tab_id'),
    "client_data": decoded_params.get('client_data'),
}

data = {
    "username" :"123",
    "password" :"123",
}

# Send the POST request with the session object
response = session.post(url, params=params, data=data, headers=headers)

# Print the status code and response body
print("Status Code:", response.status_code)
print("Response Body:", response.text)


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs

# Create a session object to handle cookies
session = requests.Session()

# Simulate the request to get the HTML response
url = "http://localhost:8080/realms/htc/protocol/openid-connect/auth"  # Replace with your actual URL
params = {
    "response_type": "code",
    "client_id": client_id,
    "state": state,
    "scope": "openid",
    "redirect_uri": redirect_uri,
    "code_challenge": code_challenge,
    "code_challenge_method": "S256",
    "response_mode": "fragment",
}

response = session.get(url, params=params)

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Extract the form's action attribute
form_action_url = soup.find('form', {'id': 'kc-form-login'})['action']

# Parse the URL to extract query parameters
parsed_url = urlparse(form_action_url)
query_params = parse_qs(parsed_url.query)

# Extract specific values
session_code = query_params.get('session_code', [None])[0]
execution = query_params.get('execution', [None])[0]
client_id = query_params.get('client_id', [None])[0]
tab_id = query_params.get('tab_id', [None])[0]
client_data = query_params.get('client_data', [None])[0]

# Print extracted values
print("session_code:", session_code)
print("execution:", execution)
print("client_id:", client_id)
print("tab_id:", tab_id)
print("client_data:", client_data)

# The cookies should be handled by the session object
cookies = session.cookies.get_dict()
print("Cookies:", cookies)

# Now, use the extracted values to simulate form submission
login_url = "http://localhost:8080/realms/htc/login-actions/authenticate"
login_data = {
    "session_code": session_code,
    "execution": execution,
    "client_id": client_id,
    "tab_id": tab_id,
    "client_data": client_data,
    "username": username,
    "password": password
}

response = session.post(login_url, data=login_data)

# Check if login was successful
if response.ok:
    print("Login successful!")
else:
    print("Login failed!")
    print("Response Text:", response.text)
